# Informed search - the A* algorithm

Depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [2]:
class Problem:
    @property
    def initial_state(self):
        ...
        
    def available_actions(self, state):
        ...        
        
    def do_action(self, state, action):
        ...
        return new_state
    
    def is_goal(self, state) -> bool:
        ...
        
    def action_cost(self, state, action) -> float:
        ...
        
    def heuristic(self, state) -> float:
        ...

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is is admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [3]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def do_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        return not any(state[1])
    
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return sum(state[1])

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a priority queue. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
  1. Pick an item with the lowest expected cost
  2. If this is the goal, return the sequence of actions necessary to reach this state
  3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.
  
Guard the algorithm against infinite loops: if you already visited a state, you don't need to visit it again (if your heuristic is consistent).

In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [111]:
from queue import PriorityQueue

def astar(problem: Problem):
    q = PriorityQueue()
    initial_state = problem.initial_state
    expected_initial_cost = problem.heuristic(initial_state)
    necessary_actions = []

    q.put((expected_initial_cost, initial_state, necessary_actions))

    previous_states = [initial_state]
    visited = 1

    while q.empty() is False:
        visited += 1
        element = q.get()
        #print("Element: ", element)

        if problem.is_goal(element[1]):
            print("Number of visited states: ", visited)
            return element[2]

        actions_list = problem.available_actions(element[1])

        for action in actions_list:
            new_state = problem.do_action(element[1], action)

            if new_state != element[1] and new_state not in previous_states:
                previous_states.append(new_state)
                cost = 0
                necessary_actions = element[2] + [action]
                expected_initial_cost = problem.heuristic(new_state)

                for a in necessary_actions:
                    cost += problem.action_cost(new_state, a)

                full_cost = cost + expected_initial_cost
                q.put((full_cost, new_state, necessary_actions))
    return None

Now lets test your code in the vacuum world!

In [113]:
astar(VacuumProblem())

Number of visited states:  5


['Suck', 'Right', 'Suck']

## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [114]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return len(state[1]) - sum(state[1])
    
astar(VacuumProblem1())

Number of visited states:  8


['Suck', 'Right', 'Suck']

And another in which heuristic grossly overestimates the cost.

In [115]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return 10 * sum(state[1])
    
astar(VacuumProblem2())

Number of visited states:  5


['Suck', 'Right', 'Suck']

**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

The only non-admissible heuristic out of those three is the third one and, surprisingly, it finds a very good solution to the VacuumProblem. That is because it favours those solutions in which the states are the closest to the goal ones and the number of actions does not matter that much (because of its' small cost in comparison to the cost of a wrong state). This problem is not complex, because only two 'elements' can be out of place and the total cost of actions will, most probably, not exceed the total cost of those elements.

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time don't change the initial state, your solution should be capable enough to solve this.

![](aima-images/fig3_4.png)

In [116]:
class PuzzleProblem(Problem):
    @property
    def initial_state(self):
        ...
        
    def available_actions(self, state):
        ...        
        
    def do_action(self, state, action):
        ...
        return new_state
    
    def is_goal(self, state) -> bool:
        ...
        
    def action_cost(self, state, action) -> float:
        ...
        
    def heuristic(self, state) -> float:
        ...

In [117]:
class PuzzleProblem(Problem):
    @property
    def initial_state(self):
        #state = [1, 0, 2, 3, 4, 5, 6, 7, 8]
        state = [7, 2, 4, 5, 0, 6, 8, 3, 1]
        return state
        
    def available_actions(self, state):
        if state[0] == 0:
            actions = ['Right', 'Down']
        elif state[1] == 0:
            actions = ['Left', 'Right', 'Down']
        elif state[2] == 0:
            actions = ['Left', 'Down']
        elif state[3] == 0:
            actions = ['Right', 'Up', 'Down']
        elif state[4] == 0:
            actions = ['Left', 'Right', 'Up', 'Down']
        elif state[5] == 0:
            actions = ['Left', 'Up', 'Down']
        elif state[6] == 0:
            actions = ['Right', 'Up']
        elif state[7] == 0:
            actions = ['Left', 'Right', 'Up']
        elif state[8] == 0:
            actions = ['Left', 'Up']
        return actions    
        
    def do_action(self, state, action):
        new_state = [x for x in state]
        zero_index = state.index(0)

        if action == "Left":
            element = new_state[zero_index - 1]
            new_state[zero_index], new_state[zero_index - 1] = new_state[zero_index-1], new_state[zero_index]
        
        elif action == "Right":
            element = new_state[zero_index + 1]
            new_state[zero_index], new_state[zero_index + 1] = new_state[zero_index+1], new_state[zero_index]
            
        elif action == "Up":
            element = new_state[zero_index - 3]
            new_state[zero_index], new_state[zero_index - 3] = new_state[zero_index-3], new_state[zero_index]
        
        elif action == "Down":
            element = new_state[zero_index + 3]
            new_state[zero_index], new_state[zero_index + 3] = new_state[zero_index+3], new_state[zero_index]
        
        else:
            raise Exception("Invalid action.")
            
        return new_state
    
    def is_goal(self, state) -> bool:
        return state == [0, 1, 2, 3, 4, 5, 6, 7, 8]
        
    def action_cost(self, state, action) -> float:
        return 1
        
    def heuristic(self, state) -> float:
        c = 0
        goal = [0, 1, 2, 3, 4, 5, 6, 7, 8]
        for x in range(len(state)):
            if state[x] != goal[x]:
                c += 1
        return c

In [122]:
astar(PuzzleProblem())

Number of visited states:  32601


['Left',
 'Up',
 'Right',
 'Down',
 'Right',
 'Down',
 'Left',
 'Left',
 'Up',
 'Right',
 'Right',
 'Down',
 'Left',
 'Left',
 'Up',
 'Right',
 'Right',
 'Up',
 'Left',
 'Left',
 'Down',
 'Right',
 'Right',
 'Up',
 'Left',
 'Left']

**Prove that this heuristic is admissible.**

This heuristic is admissible, because every tile that is out of position must be moved at least once for the puzzle to reach its goal state. That is why this heuristic guarantees that the cost will not be smaller than the actual one. It can only be greater.

In [119]:
class PuzzleProblem1(PuzzleProblem):
    def heuristic(self, state) -> float:
        goal = [0, 1, 2, 3, 4, 5, 6, 7, 8]
        numbers = []     
        for e in state:
            one = state.index(e)
            two = goal.index(e)
            #print("E: ", e, "index in beggining state: ", one, "index in goal_state: ", two)
            first = max(one, two)
            second = min(one, two)
            numbers.append(first - second)
        manhat_dist = sum(numbers)
        return manhat_dist


**Prove that this heuristic is admissible.**

This heuristic is admissible, because the manhat_dist is the sum of distances between the actual position of the tile and its goal position. In other words, it sums the minimum number of moves it is required for each tile to reach its goal. That is the reason why this number will never be smaller than the actual cost of the relocation (it is not always possible for all tiles to move directly toward their goal positions).

In [120]:
import math 
class PuzzleProblem2(PuzzleProblem):
    def heuristic(self, state) -> float:
        goal = [0, 1, 2, 3, 4, 5, 6, 7, 8]
        numbers = []
        for e in state:
            one = state.index(e)
            two = goal.index(e)
            first = max(one, two)
            second = min(one, two)
            dist = math.sqrt((first - second)^2)
            numbers.append(dist)
            final = sum(numbers)
        return final

**Prove that this heuristic is admissible.**

This heuristic is admissible, because the final is counted like the eucledian distance which is never greater than the number of the steps required for the puzzle to reach the goal state. That is the case because the tile moves by one step (and that is smaller since the square root of the number raised to the second power is smaller or equals this number).

Run your heuristics on the given problem.

In [121]:
plan0 = astar(PuzzleProblem())
plan1 = astar(PuzzleProblem1())
plan2 = astar(PuzzleProblem2())

print("Is plan0==plan1?", plan0 == plan1)
print("Is plan0==plan2?", plan0 == plan2)
print("Is plan1==plan2?", plan1 == plan2)

Number of visited states:  32601
Number of visited states:  6805
Number of visited states:  122607
Is plan0==plan1? False
Is plan0==plan2? False
Is plan1==plan2? False


**Which of the heuristics is the best for this task? Why is that?**

Out of the heuristics I have written, the best one turned out to be the one that based "the cost" on a manhattan distance of each tile (the distance between the tile and it goal position). It makes sense, because it gives an approximate number of steps required to solve the puzzle, so the algorithm will choose the next states to take into account more accurately.

------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.